<a href="https://colab.research.google.com/github/citruscaesar/building-footprint-extraction/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Dependencies

In [1]:
%%capture
!pip install wandb
!pip install lightning
!pip install torchmetrics
!pip install segmentation-models-pytorch
!pip install rasterio

In [2]:
!git clone https://github.com/citruscaesar/building-footprint-extraction.git
!mkdir -p /content/src
!cp -r /content/building-footprint-extraction/src/ /content/

Cloning into 'building-footprint-extraction'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 3), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), 1.76 MiB | 3.24 MiB/s, done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
'''
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/train /content/inria/train
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/* /content/inria/tiled
#!mkdir -p inria/tiled
'''

!mkdir -p inria/train
!cp /content/drive/MyDrive/datasets/inria/archive/tiled.zip inria/
!unzip inria/tiled.zip -d /inria/

In [5]:
from pathlib import Path
tiled = {
    "drive": Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/512x512"),
    "runtime": Path("/content/content/inria/tiled/512x512")
}
train_scenes = tiled["runtime"] / "train" / "scenes"

In [6]:
assert (len(list(train_scenes.iterdir()))) == 18600

### Imports and Config

In [7]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pytorch_lightning as pl
import torchmetrics 
from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryJaccardIndex, BinaryConfusionMatrix, BinaryF1Score, BinaryCohenKappa

import wandb
from pytorch_lightning.loggers import WandbLogger, CSVLogger

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from src.datamodule import InariaDataModule

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Pytorch Version: {torch.__version__}")
print(f"Lightning Version: {pl.__version__}")
print(f"Pytorch Running on :{DEVICE}")
print(torch.__config__.show())

Pytorch Version: 2.0.0+cu118
Lightning Version: 2.0.1
Pytorch Running on :cuda
PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.7
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS=

In [13]:
DATA = Path.cwd()/"content"/"inria"/"train" #("/content/inria/train")
DATA.mkdir(exist_ok=True)
#DATA = Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/train")
#DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/inaria/train")

### Segmentation Module

In [14]:
pl.seed_everything(seed = 69, workers = True)

INFO:lightning_fabric.utilities.seed:Global seed set to 69


69

In [15]:
class SegModule(pl.LightningModule):
    def __init__(self, model, preprocess, learning_rate, batch_size):
        super().__init__()
        self.model = model
        self.preprocess_fn = preprocess
        self.loss = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits = True)

        self.save_hyperparameters(ignore = ["model", "preprocess"])

        self.val_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Validation ")

        self.test_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Test ")


    def forward(self, batch):
        mask = self.model(batch) 
        return mask
        
    def training_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask) 
        self.log("train/loss", loss.item(), on_epoch = True, on_step=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("val/loss", loss.item(), on_epoch=True)

        self.val_metrics.update(pred_mask, mask)
        self.log_dict(self.val_metrics, on_epoch=True)

    def test_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("test/loss", loss.item(), on_epoch=True)

        self.test_metrics.update(pred_mask, mask)
        self.log_dict(self.test_metrics, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr = self.hparams.learning_rate)
        return optimizer

### Experiments

In [ ]:
#config dict

In [24]:
#os.environ["WANDB_NOTEBOOK_NAME"] = "experiments"
wandb.finish()
logs_dir = Path.cwd() / "logs"
logs_dir.mkdir(exist_ok = True)
wandblogger = WandbLogger(
    project = "bulding-segmentation-tests",
    save_dir = logs_dir 
) 

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find experiments.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [18]:
unet = smp.Unet("resnet18", encoder_weights="imagenet", in_channels=3, classes = 1)
preprocess = get_preprocessing_fn("resnet18", "imagenet")

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 135MB/s]


In [26]:
model = SegModule(unet, preprocess, 3e-4)

In [55]:
dm = InariaDataModule(DATA, batch_size = 16, num_workers = 2)

In [57]:
dev_trainer = pl.Trainer(
    accelerator = "auto",
    fast_dev_run = 1
)

trainer = pl.Trainer(
    accelerator = "auto",
    #accumulate_grad_batches = 2,
    #limit_train_batches = .25,
    max_epochs = 1,
    precision = 16,
    #logger = [wandblogger],
    #check_val_every_n_epochs = 1
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [58]:
#dev_trainer.fit(model, datamodule = dm)

In [59]:
trainer.fit(model, datamodule = dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params
--------------------------------------------------
0 | model        | Unet             | 14.3 M
1 | loss         | DiceLoss         | 0     
2 | val_metrics  | MetricCollection | 0     
3 | test_metrics | MetricCollection | 0     
--------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [54]:
trainer.test(model, datamodule = dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   Test BinaryCohenKappa   │    0.8268173933029175     │
│    Test BinaryF1Score     │    0.8509175777435303     │
│  Test BinaryJaccardIndex  │    0.7405191659927368     │
│         test/loss         │    0.15037842094898224    │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.15037842094898224,
  'Test BinaryF1Score': 0.8509175777435303,
  'Test BinaryJaccardIndex': 0.7405191659927368,
  'Test BinaryCohenKappa': 0.8268173933029175}]

In [60]:
wandb.finish()

Validation BinaryCohenKappa,▁
Validation BinaryF1Score,▁
Validation BinaryJaccardIndex,▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_epoch,▁
train/loss_step,█▆▅▄▄▂▂▂▃▂▁▂▂▂
trainer/global_step,▁▂▃▄▄▁▂▃▄▄▅▆▇███
val/loss,▁
Validation BinaryCohenKappa,0.82327
Validation BinaryF1Score,0.85191
Validation BinaryJaccardIndex,0.74202


In [61]:
drive.flush_and_unmount()